In [1]:
import pandas as pd

from common.path import COMMON_PATH

df = pd.read_csv(COMMON_PATH/'folder_possibles.csv')
dfex = pd.read_excel(COMMON_PATH/'bdd_SIRET.xlsx')
dfbase = dfex[~dfex.folder.isin(df.folder.values)]
dfbase = dfbase.set_index('folder')

logfile available C:\Users\lvolat\Documents\cocoAI\work\ipykernel_launcher.log


In [2]:
from common.path import COMMERCIAL_DOCUMENTS_PATH

# j ai pas pris les hotels
# j ai pas pris l'immobilier
# C:\Users\lvolat\COMPTOIRS ET COMMERCES\COMMERCIAL - Documents\2 - DOSSIERS à l'ETUDE\4 - IMMOBILIER
# j ai pas pris 3 - ACHETEURS & GERANTS LIBRES
# j ai pas pris 4 - SUIVI DES APPORTS AFFAIRES
# j ai pas pris 100 - DOSSIERS ARCHIVÉS\2 - ARCHIVES DOSSIERS SCANNÉS
# j ai pris les dossiers archives


dossiers = (
    [d.name for d in list(COMMERCIAL_DOCUMENTS_PATH.glob("1 -*/*/"))]
    + [d.name for d in list(COMMERCIAL_DOCUMENTS_PATH.glob("2*/1*/*/"))]
    + [d.name for d in list(COMMERCIAL_DOCUMENTS_PATH.glob("10 -*/*/"))]
    + [d.name for d in list(COMMERCIAL_DOCUMENTS_PATH.glob("100 -*/1 -*/*/"))]
)
dossiers = [d for d in dossiers if d not in df.folder.values]
print(len(dossiers),dossiers)

40 ['2BC AUBRAC - BORREL BONNENFANT', 'LE DIPLOMATE - Place SINGER - 75016 PARIS', 'LOUIS XVI -  75008 PARIS\xa0- 47 Rue des MATHURINS', 'MONTCASTEL SARL - Patrick NONNET', 'WOOD - LE BISTROT À VINS  - 92400 COURBEVOIE - 35 Place des COROLLES', "BUISSON D'ARGENT - Rue du Bac - 75007 Paris", "CAFE DE L'OLYMPIA - 2 Rue de SÈZE - 75009 PARIS", 'CAFE DE LA PLACE - 52 Rue de Montreuil - 94 Vincennes', 'CAFE GALLERY - 78 Rue de Provence - 75009 Paris', 'CARRE ELYSÉE - 49 rue Franklin D. Roosevelt - 75008 Paris', 'COMPTOIR BROCHANT - 47 Rue BROCHANT - 75017 PARIS', 'DEPART (LE) - 1 Rue du ROCHER - 75008 PARIS', 'GYMNASE (LE) -  208 Bld RASPAIL - 75014 PARIS', 'LOUVIERE (LA)', 'MARCEAU - Av MARCEAU - 75016 PARIS', "PATISSERIE DE L'OLYMPIA - 2 Rue de SÈZE - 75009 PARIS", 'PINOCCHIO (LE) - 19 Bld Edgar QUINET - 75014 PARIS', 'POULE AU POT', 'TELEPHONES - Rue CLER - 75007 PARIS', 'TERRASSE 17 - 17 Rue des BATIGNOLLES - 75017 PARIS', "TERRASSE DU 7 - 2 Place de l'ÉCOLE MILITAIRE - 75007 PARIS", 'T

In [15]:
print([d[7:] for d in dossiers[30:40]])
print('pour la liste suivante trouve les SIRET de chaque établissement. Présente le résultat sous forme suivante : df[établissement,"'"SIRET"'"] = SIRET')


['RAC - BORREL BONNENFANT', 'OMATE - Place SINGER - 75016 PARIS', 'VI -  75008 PARIS\xa0- 47 Rue des MATHURINS', 'TEL SARL - Patrick NONNET', 'LE BISTROT À VINS  - 92400 COURBEVOIE - 35 Place des COROLLES', " D'ARGENT - Rue du Bac - 75007 Paris", " L'OLYMPIA - 2 Rue de SÈZE - 75009 PARIS", ' LA PLACE - 52 Rue de Montreuil - 94 Vincennes', 'LLERY - 78 Rue de Provence - 75009 Paris', 'LYSÉE - 49 rue Franklin D. Roosevelt - 75008 Paris']
pour la liste suivante trouve les SIRET de chaque établissement. Présente le résultat sous forme suivante : df[établissement,"SIRET"] = SIRET


In [4]:
import re
import numpy as np


def get_enseigne(dossier):
    if "EN COURS" in dossier:
        dossier = dossier.replace("EN COURS", "")
    if re.match(r"\d{4} .*", dossier):
        dossier = "-".join(dossier.split("-")[1:])

    dossier = dossier.strip().strip("-")
    sp = dossier.split("-")
    if len(sp) == 1:
        enseigne = sp[0]
        CP = np.nan
        ville = np.nan
        adresse = np.nan
    elif len(sp) == 2:
        enseigne = sp[0]
        CP = np.nan
        ville = np.nan
        adresse = np.nan
    elif len(sp) == 3:
        enseigne = sp[0]
        adresse, CP, ville = detect_adresse(sp[1], sp[2])
    elif len(sp) == 4:
        enseigne = "-".join([sp[0], sp[1]])
        adresse, CP, ville = detect_adresse(sp[2], sp[3])
    else:
        print(dossier)
        raise ValueError("NotImplemented")
    # print(enseigne,adresse,CP,ville)
    # try:
    return (
        str(enseigne.strip()),
        adresse.strip() if isinstance(adresse, str) else adresse,
        int(CP) if not np.isnan(float(CP)) else CP,
        ville,
    )

def detect_adresse(m1, m2):
    # print(m1,m2)
    if len(m1.split()) == 2 and re.match(r"\d{5} .*", m1.strip()):
        CP = m1.split()[0]
        ville = m1.split()[1]
        adresse = m2
    else:
        CP = m2.split()[0]
        ville = m2.split()[1]
        adresse = m1.strip()
    return adresse, CP, ville


for d in dossiers:
    enseigne, adresse, CP, ville = get_enseigne(d)
    ser = pd.Series(
        {"ENSEIGNE": enseigne, "ADRESSE": adresse, "CP_x": CP, "VILLE": ville}, name=d
    )
    dfbase.loc[d, :] = ser
    # dfbase = pd.concat([dfbase,ser],axis=1)
    # print(d)
    # get_enseigne(d)
    # print(d, get_enseigne(d))

dfbase["CP"] = dfbase["CP_x"].apply(
    lambda x: str(int(x)) if not np.isnan(x) else np.nan
)

dfbase = dfbase.dropna(axis=1,how='all')
del dfbase["CP_x"]
# del dfbase["CP_y"]
# del dfbase["Unnamed: 0"]
dfbase.loc[:, "SIRET"] = np.nan
dfbase.reset_index(inplace=True)
dfbase.to_csv("tmp.csv")

In [18]:
print(dfbase.folder.values.tolist()[10:20])
print("pour la liste suivante trouve les SIRET de chaque établissement. Présente le résultat sous forme d'un dictionnaire json  dont les clés sont les membres de la liste et les valeurs les numéros de SIRET")

['COMPTOIR BROCHANT - 47 Rue BROCHANT - 75017 PARIS', 'DEPART (LE) - 1 Rue du ROCHER - 75008 PARIS', 'GYMNASE (LE) -  208 Bld RASPAIL - 75014 PARIS', 'LOUVIERE (LA)', 'MARCEAU - Av MARCEAU - 75016 PARIS', "PATISSERIE DE L'OLYMPIA - 2 Rue de SÈZE - 75009 PARIS", 'PINOCCHIO (LE) - 19 Bld Edgar QUINET - 75014 PARIS', 'POULE AU POT', 'TELEPHONES - Rue CLER - 75007 PARIS', 'TERRASSE 17 - 17 Rue des BATIGNOLLES - 75017 PARIS']
pour la liste suivante trouve les SIRET de chaque établissement. Présente le résultat sous forme d'un dictionnaire json  dont les clés sont les membres de la liste et les valeurs les numéros de SIRET


In [20]:
di = {
    "LE DIPLOMATE - Place SINGER - 75016 PARIS": "83389469400011",
    "2BC AUBRAC - BORREL BONNET": "79903742900011",
    "LOUIS XVI -  75008 PARIS 47 Rue des MATHURINS": "50528188100027",
    "MONTCASTEL SARL - Patrick NONNET": "82044311700016",
    "WOOD - LE BISTROT À VINS  - 92400 COURBEVOIE - 35 Place des COROLLES": "40753333000012",
    "BUISSON D'ARGENT - Rue du Bac - 75007 Paris": "56201528900016",
    "CAFE DE L'OLYMPIA - 2 Rue de SÈZE - 75009 PARIS": "34027633600039",
    "CAFE DE LA PLACE - 52 Rue de Montreuil - 94 Vincennes": "34507812500011",
    "CAFE GALLERY - 78 Rue de Provence - 75009 Paris": "None",
    "CARRE ELYSÉE - 49 rue Franklin D. Roosevelt - 75008 Paris": "71202157500015",
    "TERRASSE DU 7": "44348736800013",
    "TOURNON (LE)": "42294507100018",
    "VENICE": "38914469200057",
    "COMPTOIR BROCHANT - 47 Rue BROCHANT 75017 PARIS": "53258418200019",
    "DEPART (LE) - 1 Rue du ROCHER 75008 PARIS": "33443868600019",
    "GYMNASE (LE) - 208 Bld RASPAIL 75014 PARIS": "31677872900012",
    "LOUVIERE (LA)": "47150251800018",
    "MARCEAU - Av MARCEAU 75016 PARIS": "45229220400019",
    "PATISSERIE DE L'OLYMPIA - 2 Rue de SÈZE 75009 PARIS": "81369267000025",
    "PINOCCHIO (LE) - 19 Bld Edgar QUINET 75014 PARIS": "90950914300022",
    "POULE AU POT": "44175867900019",
    "TELEPHONES - Rue CLER 75007 PARIS": "90950914300022",
    "TERRASSE 17 - 17 Rue des BATIGNOLLES 75017 PARIS": "98247515400018",
    "VILLA SOPHIA": "44233251600016",
    "ZIG ZAG CAFE": "48913649900014",
    "BEY STEACK HOUSE": "91277635800018",
    "COQUELICOT DES ABBESSES": "44233251600016",
    "ALFIO": "91276108400014",
    "FOISONNANTE": "85016864200013",
    "GOLFE DE NAPLES": "32165226500018",
    "2023 - KIBALOMA - PARIS PYLA - 4 Place du 18 JUIN 1940 - 75006 PARIS": "78419152000047",
    "2023 - SCORZO - 64 Avenue des TERNES - 75017 PARIS": "38809476500020",
    "2023 - SUFFREN - 84 Av de SUFFREN – 75015 PARIS": "30376703200012",
    "2023 - THE JUNGLE - 4 Rue CAULAINCOURT - 75018 PARIS - EN COURS": "Non trouvé",
    "2023 _ 2024 - GAULOISE - 59 Av de la MOTTE PICQUET - 75015 PARIS": "30176296900016",
    "2024 - BERTHOUD (LE) - 1 Rue VALETTE - 75005 PARIS": "Non trouvé",
    "2024 - CAFE DES BEAUX ARTS (LE) - 7 Quai MALAQUAIS - 75006 PARIS": "83394006700011",
    "2024 - EUGENE EUGENE - 38-40 Rue Eugène EICHENBERGER - 92800 PUTEAUX": "80224059800010",
    "2024 - MARCELLO - 10 Bld DELESSERT - 75016 PARIS": "Non trouvé",
    "2025 - PETIT BISTROT D'AUTEUIL (Le) - 75016 PARIS - 8 Bis Rue VEEDERET": "Non trouvé",
}

In [21]:
len(di.keys())

40

In [26]:
print(min([len(d) for d in di.keys()]))

5


In [31]:
import pandas as pd


# Fonction pour mapper les valeurs
def mapper_valeurs(valeur):
    prefixe = valeur[:10]
    for k in di.keys():
        if k[:10] == prefixe:
            return di.get(k, None)

# Création de la nouvelle colonne 'SIRET'
dfbase['SIRET'] = dfbase['folder'].apply(mapper_valeurs)

# print(dfbase)


In [32]:
dfbase

,folder,ENSEIGNE,VILLE,ADRESSE,CP,SIRET
0,LE DIPLOMATE - Place SINGER - 75016 PARIS,LE DIPLOMATE,PARIS,Place SINGER,75016,83389469400011
1,2BC AUBRAC - BORREL BONNENFANT,2BC AUBRAC,NaN,NaN,NaN,79903742900011
2,LOUIS XVI - 75008 PARIS - 47 Rue des MATHURINS,LOUIS XVI,PARIS,47 Rue des MATHURINS,75008,50528188100027
3,MONTCASTEL SARL - Patrick NONNET,MONTCASTEL SARL,NaN,NaN,NaN,82044311700016
4,WOOD - LE BISTROT À VINS - 92400 COURBEVOIE -...,WOOD - LE BISTROT À VINS,COURBEVOIE,35 Place des COROLLES,92400,40753333000012
5,BUISSON D'ARGENT - Rue du Bac - 75007 Paris,BUISSON D'ARGENT,Paris,Rue du Bac,75007,56201528900016
6,CAFE DE L'OLYMPIA - 2 Rue de SÈZE - 75009 PARIS,CAFE DE L'OLYMPIA,PARIS,2 Rue de SÈZE,75009,34027633600039
7,CAFE DE LA PLACE - 52 Rue de Montreuil - 94 Vi...,CAFE DE LA PLACE,Vincennes,52 Rue de Montreuil,94,34507812500011
8,CAFE GALLERY - 78 Rue de Provence - 75009 Paris,CAFE GALLERY,Paris,78 Rue de Provence,75009,None
9,CARRE ELYSÉE - 49 rue Franklin D. Roosevelt - ...,CARRE ELYSÉE,Paris,49 rue Franklin D. Roosevelt,75008,71202157500015


In [ ]:
# dfbase.to_csv('tmp2.csv')

In [40]:
from common.path import WORK_PATH


dfbase = pd.read_csv(WORK_PATH / "tmp2.csv", index_col=0,sep=';')
dfbase = dfbase.rename({'SIRET':'siret'},axis=1)
df = pd.read_csv(COMMON_PATH / "folder_possibles.csv", index_col=0)
df["siret"] = df["siret"].apply(
    lambda x: str(int(x)) if not np.isnan(float(x)) else np.nan
)

In [46]:
df.columns

Index(['folder', 'ENSEIGNE', 'Unnamed: 0', 'ville', 'adresse', 'CP_x',
       'TYPE DE VENTE', 'RAISON SOCIAL', 'EN VENTE', 'GÉRANCE LIBRE',
       'ADRESSE ', 'CP_y', 'VILLE', 'ADRESSE', 'AFFAIRE', 'Colonne1', 'siret'],
      dtype='object')

In [47]:
df2 = pd.concat([df,dfbase])

In [49]:
df2.to_csv(COMMON_PATH / "folder_possibles_complet.csv")